## Notes

### This model grabs Pitch and Timbre

#### Project Progress :

1. ~~find zero-crossings~~
2. ~~find cycles~~
3. ~~choose representative cycles based on biases~~
4. form a model with bcoeffs for each representative cycle
5. interpolate between each cycle using bcoeffs for amplitude shape and glissando for pitch
6. test accuracy to original sound
7. Tokenize and train model
8. Profit?
   
#### Ideas

Test with no interpolation between the representative cycles or by sample-and-hold the cycles for a duration

What if instead of tokenizing the b-spline coefficients we use the overtones or harmonics of each cycle? At the end of the day, we know that a neural network can distinguish pitch visually.

What if we tokenize the information of the differencs in the b-splines

To better find cycles: group cycles by similarity by using a dot product, larger dot products are much more similar. Find one from the largest group as your repcycle. Make a Union of the groups to find the group whose cycles covers the whole segment.

Compare cycles of next segment with the repcycle of the previous 

If the segment has 30+ cycles in its segment, it might be noise therfore bypass the 1st 3rd bias 

Instead of using Bspline coefficients, just use vectorized versions

Make custom FFT that limits bins to only range we need, with optential peek ahead to see dominant frequencies over

Todo RECONSTRUCT SAMPLE-AND-HOLD VECTORIZED

TODO create commandline process to make repcycles and of the dataset based on parameters

In [1]:
# Testing for Pytorch Audio prcessing
# Andrei Cartera 
import os, sys
import numpy as np
#from pathlib import Path
import matplotlib.pyplot as plt
import torchaudio.transforms as T
import torch.nn.functional as nnF
#from IPython.display import Image
import heapq
import math

import torch
import torchaudio 

# Install soundfile and ffmpeg-python if not already installed
# pip install soundfile
# pip install ffmpeg-python

print(torch.__version__)
print(str(torchaudio.list_audio_backends()))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

ModuleNotFoundError: No module named 'numpy'

In [ ]:
AUDIO_PATH = "./audio/"
FILE_NAME = "one.wav"
AUDIO_SIZE = 16000
SEGMENT_LENGTH = 500
SEGMENT_NUM = 5
NOISE_FACTOR = 0.65
CYCLE_EPSILON = 30 # make it percentage wise 25%
FFT_SIZE = 1024

REPC_VEC_SIZE = 64
FRONT_VEC_SIZE = 40
Z_THRESHOLD = 1.0
DIFF_THRESHOLD = 0.3

OPEN_WAV = True
OPEN_FFT = False
OPEN_REPC = True
SHOW_ZEROS = True

WAV_FIG_SIZE = (50, 10)
FFT_FIG_SIZE = (40, 10)
F0_FREQ_RANGE = (50,250)

# how much to conisder frequency, loudness, or middleness, higher values means more adherence
RMS_WEIGHT    = 1 # RMS loudness
FREQ_WEIGHT   = 1 # Frequency 
MID_WEIGHT    = 2 # Middle

In [ ]:
# DO NOT EDIT
LOW_F0_INDEX = math.floor(FFT_SIZE/AUDIO_SIZE * F0_FREQ_RANGE[0])
HIGH_F0_INDEX = math.floor(FFT_SIZE/AUDIO_SIZE * F0_FREQ_RANGE[1])

### Helper Functions and Misc

In [ ]:
def wav_value(waveform_arr, location:float):
    # Ensure the location is within the bounds of the array
  if location < 0 or location > len(waveform_arr) - 1:
    raise ValueError("Location is out of bounds.")

  # Define the indices for interpolation
  indices = np.arange(len(waveform_arr))

  # Use numpy's interp function to get the interpolated value
  return np.interp(location, indices, waveform_arr)

def vectorize(waveform_arr, n:int):
  indices = np.linspace(0, len(waveform_arr) - 1, n, dtype=int)
  return waveform_arr[indices]

def vectorize_f(waveform_arr, n:int):
  if len(waveform_arr) == n:
    return waveform_arr
  if len(waveform_arr) < n:
    # resample the waveform to fit the desired size
    return resample(waveform_arr, n)
    # pad with zeros
    #return np.pad(waveform_arr.numpy(), (0,n - len(waveform_arr)))
  
  sample_points = np.linspace(0, len(waveform_arr) - 1, n)
  return np.array([wav_value(waveform_arr, point) for point in sample_points])

def resample(waveform_arr, n:int):
  resampled_waveform = np.zeros(n)
  sample_points = np.linspace(0, len(waveform_arr) - 1, n)
  for i, point in enumerate(sample_points):
    resampled_waveform[i] = wav_value(waveform_arr, point)
  return resampled_waveform

def bias_front(waveform_arr):
  front = (len(waveform_arr) - 1) // 3
  #front = int((len(waveform_arr) - 1) // (f0//(F0_FREQ_RANGE[0]//2)))
  #adds up 1st 3rd of the
  return np.sum(waveform_arr[:front])

def bias_front_abs(waveform_arr):
  front = (len(waveform_arr) - 1) // 3
  #front = int((len(waveform_arr) - 1) // (f0//(F0_FREQ_RANGE[0]//2)))
  #adds up 1st 3rd of the
  return np.sum(abs(waveform_arr[:front]))
  
# gets k largest elements from a list along with its indices 
def klargest_with_indices(arr, k, range=None): 
  heap = []
  for i, val in enumerate(arr):
      if range and not (range[0] <= i <= range[1]):
        continue
      heap.append((i,val))
  return heapq.nlargest(k, heap, key=lambda x: x[1])

def calculate_rms(audio):
  return np.sqrt(np.mean(audio**2))

def find_factor_pairs(n):
  factor_pairs = []
  for i in range(1, int(n**0.5) + 1):
    if n % i == 0:
      factor_pairs.append((i, int(n // i)))
  return factor_pairs

#def group_tuples(tuples, range_size):
#  # Sort tuples based on the second element
#  tuples_sorted = sorted(tuples, key=lambda x: x[1])  
#  
#  # Group tuples based on the range_size
#  groups = []
#  current_group = []
#  for tup in tuples_sorted:
#    if not current_group or abs(tup[1] - current_group[-1][1]) <= range_size:
#        current_group.append(tup)
#    else:
#      groups.append(current_group)
#      current_group = [tup]
#  if current_group:
#    groups.append(current_group)
#  
#  return groups

# returns majority and outliers based on z-scores for the specified element in a list tuples
def distinguish_outliers_by_front(tuples, z_threshold=1.0):
  # Extract the second element from each tuple
  second_terms = np.array([tup[1] for tup in tuples])
  
  # Calculate mean and standard deviation
  mean = np.mean(second_terms)
  std_dev = np.std(second_terms)
  
  if std_dev == 0:
    # If all second terms are identical, classify all as "majority" and no "outliers"
    return tuples, []
  # Calculate z-scores for each second element
  z_scores = (second_terms - mean) / std_dev
  
  # Classify tuples into "majority" and "outliers"
  majority = [tuples[i] for i in range(len(tuples)) if abs(z_scores[i]) <= z_threshold]
  outliers = [tuples[i] for i in range(len(tuples)) if abs(z_scores[i]) > z_threshold]
  
  return majority, outliers, mean

# returns an array of RMS values from a signal over time specified by the window size 
def rms_over_windows(waveform_arr, window_size=100, silence_threshold = None):
  rms_values = []
  excluded_ranges = []
  
  start = 0
  while start < len(waveform_arr):
    end = start + window_size
    if end > len(waveform_arr):
      end = len(waveform_arr)
    window = waveform_arr[start:end]
    rms_value = calculate_rms(window)
    rms_values.append(rms_value)
    
    if silence_threshold:
      if rms_value < silence_threshold:
        excluded_ranges.append((start, end))

    start = end
  
  return rms_values, excluded_ranges

def fft_max(waveform_arr : np.array):

  # Pad with zeros if necessary 
  if(len(waveform_arr) != FFT_SIZE):
    size_diff = abs(len(waveform_arr)-FFT_SIZE)
    list(waveform_arr).extend([0] * size_diff)

    # perform fft and get magnitude
  fft = np.fft.fft(waveform_arr, FFT_SIZE)
  fft_mag = np.abs(fft)
  
    #turn back to np.array
  fft_mag = fft_mag[:FFT_SIZE // 2]
  max_index = np.argmax(fft_mag)
  return fft_mag, max_index

def show_fft(fft_mag, start_sample:int, top:int=0):
  from matplotlib.ticker import ScalarFormatter
  
  # make x-axis frequencies
  frequencies = np.linspace(0, AUDIO_SIZE / 2, FFT_SIZE // 2)
  out_path = f".\\figures\\fft_{FILE_NAME.split('.')[0]}_{start_sample}.png"
  # Convert magnitude to dB
  #fft_dB = 20 * np.log10((2/SEGMENT_LENGTH) * fft_mag)

  # Plot the FFT magnitude as a bar plot
  fig = plt.figure(figsize=FFT_FIG_SIZE)
  ax = fig.add_subplot(111)
  #plt.plot(frequencies, fft_dB)
  plt.bar(frequencies, fft_mag, width=15, align='center', alpha=0.75)
  plt.xscale('log')
  plt.xticks([50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000, 3000, 4000, 5000, 8000 ],
             ['50','100','200','300','400','500','600','700','800','900','1000','1500','2000','3000','4000','5000','8000']) 
  plt.xticks(rotation=45)  # Rotate by 45 degrees
  plt.gca().xaxis.set_major_formatter(ScalarFormatter())
  plt.title(f"FFT of \"{FILE_NAME}\" from {start_sample} to {start_sample+SEGMENT_LENGTH} samples")
  plt.xlabel("Frequency (Hz)")
  plt.ylabel("Magnitude")
  if(top):
    largest = klargest_with_indices(fft_mag, top, (LOW_F0_INDEX, HIGH_F0_INDEX))
    for index,value in largest:
      max_x = AUDIO_SIZE/FFT_SIZE * index
      max_y = value
      ax.plot(max_x, max_y, 'ro')
      ax.annotate(f'{max_x} Hz', xy=(max_x, max_y), xytext=(max_x+max_x/15, max_y),color='red')
  plt.grid(True)
  plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
  plt.close()
  # Open the saved image with the default viewer
  if OPEN_FFT:
    if os.path.exists(out_path):
      if os.name == 'nt':  # For Windows
        os.startfile(out_path)

  # Plot the waveform
def show_waveform(waveform_arr, start_sample, zero_crossings:np.array=None, repcycle=None, filename=None):
  
  out_path = f".\\figures\\{FILE_NAME.split('.')[0]}_{start_sample}.png"
  if filename:
    out_path = f".\\figures\\{filename}.png"

  fig = plt.figure(figsize=WAV_FIG_SIZE)
  ax = fig.add_subplot(111)
  plt.plot(range(start_sample, start_sample+len(waveform_arr)), waveform_arr)
  y_max = max(abs(np.amin(waveform_arr)), abs(np.amax(waveform_arr)))

  if y_max < 0.1: 
    y_max = 0.1
    plt.ylim(-y_max, y_max)
  if zero_crossings is not None and zero_crossings.size > 0 and SHOW_ZEROS:
    plt.vlines(zero_crossings, -y_max, y_max, colors='red', linestyles="dashed", linewidth=1)

  if(repcycle):
    #ax.fill_between(t, 1, where=s > 0, facecolor='green', alpha=.5)
    #ax.fill_between(t, -1, where=s < 0, facecolor='red', alpha=.5)
    ax.fill_between(repcycle, -y_max, y_max, facecolor='green', alpha=.25)
  plt.xticks(np.arange(start_sample, start_sample+len(waveform_arr), 100))
  plt.title(f"Waveform of \"{FILE_NAME}\"")
  plt.xlabel("Samples")
  plt.ylabel("Amplitude")
  plt.grid()
  plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
  plt.close()
  # Open the saved image with the default viewer
  if OPEN_WAV:
    if os.path.exists(out_path):
      if os.name == 'nt':  # For Windows
        os.startfile(out_path)
  
def show_repcycles(repcycles, waveform_arr, vectorize=False):
  # Find a pair of multiples of n_segments for dysplaing grid of waveforms 
  n_segments = AUDIO_SIZE / SEGMENT_LENGTH
  min_diff = np.inf
  row_col_pair = None
  for pair in find_factor_pairs(n_segments):
    diff = abs(pair[0]-pair[1])
    if(diff < min_diff):
      row_col_pair = pair
      min_diff = diff
  nrows, ncols = row_col_pair

  ### make grid plot
  _, axes = plt.subplots(nrows, ncols, figsize=(100, 50))

  axes = axes.flatten()
  # Loop through the data and corresponding axes to plot
  for i, ax in enumerate(axes):
    if i < len(repcycles):

      if repcycles[i]:
        
        curr_repcycle = repcycles[i]
        repc_start = math.floor(curr_repcycle[0])
        repc_end = math.floor(curr_repcycle[1])
        ticks = np.linspace(repc_start, repc_end, 10)
        repcycle_wav = waveform_arr[repc_start:repc_end]
        if vectorize:
          ax.plot(np.linspace(repc_start,repc_end, REPC_VEC_SIZE), vectorize_f(repcycle_wav, REPC_VEC_SIZE))
        else:
          ax.plot(range(repc_start, repc_end), repcycle_wav)
        ax.set_xticks(ticks)
        y_max = max(abs(np.amin(repcycles[i])), abs(np.amax(repcycles[i])))   

        if y_max < 0.1: 
          y_max = 0.1
          plt.ylim(-y_max, y_max)

        ax.grid(True)

      else:
        ax.text(0.5, 0.5, 'NONE', fontsize=50, ha='center', va='center', alpha=0.5)
    else:
      ax.axis('off') 

  out_path = f".\\figures\\{FILE_NAME}_repc_segmentsize{SEGMENT_LENGTH}.png"

  plt.tight_layout()
  plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
  plt.close()
  # Open the saved image with the default viewer
  if OPEN_REPC:
    if os.path.exists(out_path):
      if os.name == 'nt':  # For Windows
        os.startfile(out_path)

In [ ]:
def find_zerocrossings(waveform_arr:np.array, start_sample, dynamic_threshold=0.01):
  last=None 
  zerocrossings = np.array([], dtype='f')
  
  #Filter out quiet parts of the signal using a RMS with windowing
  _, excluded_ranges = rms_over_windows(waveform_arr, SEGMENT_LENGTH//5, dynamic_threshold)

  # turn the excluded ranges into an array of samples that shouldn't be processed
  excluded_samples = [] 
  for start, end in excluded_ranges:
    excluded_samples.extend(range(start_sample + start, start_sample + end))

  #sweep the whole segment
  for index, value in enumerate(waveform_arr):
    absolute_index = start_sample + index
    if absolute_index in excluded_samples:
      continue 
    if last == None:  #record last sample
      last = value
    #elif value == 0:
    #  if np.sign(last) == -1.0 and np.sign(waveform_arr[index+1]) == 1.0:
    #    np.append(zerocrossings, start_sample+index)
    
    # if last sample is negative and current one is positive a zero crossing occured
    if np.sign(last) == -1.0 and np.sign(value) == 1.0: 
      # x = x1 + (x2-x1)/(y2-y1)*(y-y1)
      inter_x = (index-1) + (-last/(value-last))
      zerocrossings = np.append(zerocrossings,start_sample+inter_x)
    last = value
  return zerocrossings

In [ ]:
def find_cycles_f0(f0, start_sample, zero_crossings:list):
  
  cycles = []
  f0_length = 1.0/f0 * AUDIO_SIZE #length of samples of fundamental frequency
  last_sample = start_sample*SEGMENT_LENGTH + SEGMENT_LENGTH 
  for start in zero_crossings:
    cyclef0_end = start+f0_length # where we are expecting the cycle to end 
    
    if cyclef0_end > last_sample+CYCLE_EPSILON: #don't look past end of segment, unless CYCLE_EPSILON allows you to see the end 
      continue

    high = cyclef0_end + CYCLE_EPSILON #acceptabale ranges
    low = cyclef0_end - CYCLE_EPSILON
    
    #get values that fall withing the f0+epsilon range
    try:
      possible_cycles = list(filter(lambda x, high=high, low=low: low <= x <= high ,zero_crossings))
      if not possible_cycles:
        #print(f"Failed to find a cycle for sample {start} with an epsilon of {CYCLE_EPSILON}")
        continue
      if len(possible_cycles) > 1:
        # tie-breaker get the end sample that is closest to the expected
        differences = list(map(lambda x, y=cyclef0_end: abs(y-x), possible_cycles))
        end = possible_cycles[np.argmin(differences)]
      else:
        end = possible_cycles[0]
    except Exception as e:
      print(f"Error: Failed to find a cycle for sample {start} with an epsilon of {CYCLE_EPSILON}\n",e)
    cycles.append((start,end))
  return cycles
      

In [ ]:
def find_repcycle(segment_wav, start_sample, f0, cycles):
  # 2. closeness to f0
  # 3. loudness
  # 4. most in the middle
  
  mid_segment = start_sample+ (SEGMENT_LENGTH/2);
  f0_length = 1.0/f0 * AUDIO_SIZE #length of samples of fundamental frequency
  
  cycles_data = [] # (index, freq_score, rms_score, middle_score)
  for index, cycle in enumerate(cycles):
    
    #get length of the cycle and comapre with fundamental frequency reciprocal 
    freq = cycle[1] - cycle[0]
    f_score = abs(freq - f0_length)
    
    #get middle and comare with middle of the whole segment  
    middle = (cycle[0] + cycle[1]) / 2
    m_score = abs(middle - mid_segment)

    #get rms
    i_beginning = math.floor(cycle[0]-start_sample)
    i_end = math.floor(cycle[1]-start_sample)
    cyc_wav = segment_wav[i_beginning:i_end]
    rms = calculate_rms(cyc_wav)
    r_score = 1-rms

    #identify cycles that have an obvious shape at the beginning of the cycle by using bias_front

    #cyc_wavs.append(cyc_wav)
    cycles_data.append((index, f_score, r_score, m_score))
    
  # Normalize scores
  max_f_score = max(cycles_data, key=lambda x: x[1])[1]
  max_r_score = max(cycles_data, key=lambda x: x[2])[2]
  max_m_score = max(cycles_data, key=lambda x: x[3])[3]

  normalized_cycles_data = [
    (index, f_score / max_f_score, r_score / max_r_score, m_score / max_m_score)
    for index, f_score, r_score, m_score in cycles_data
  ]

  # Apply weights and sort
  weighted_cycles_data = [
    (index, f_score * FREQ_WEIGHT, r_score * RMS_WEIGHT, m_score * MID_WEIGHT)
    for index, f_score, r_score, m_score in normalized_cycles_data
  ]

    #loop throuh all combination of cycles to take the dot product of all, forward only
  #for cyc1_index, cyc1 in enumerate(cyc_wavs):
  #  if cyc1_index > len(cyc_wavs)-1:
  #    break
  #  for j, cyc2 in enumerate(cyc_wavs[cyc1_index+1:]):
  #    cyc2_index = j+cyc1_index+1
  #    
  #    #pad with zeroes if cycles are not same size
  #    if len(cyc1) < len(cyc2):
  #      cyc1 = np.pad(cyc1,(0,len(cyc2)-len(cyc1)))
  #    elif len(cyc2) < len(cyc1):
  #      cyc2 = np.pad(cyc2,(0,len(cyc1)-len(cyc2)))
  #    cyc_dots.append(np.dot(cyc1,cyc2), cyc1_index, cyc2_index) # (dot_prod, cyc1_index , cyc2_index) 
  
  
  
  sorted_list = sorted(weighted_cycles_data, key=lambda x: x[1]+x[2]+x[3])
  #assert sorted_list != [], f"No repcycles found at {start_sample}"
  if sorted_list == []:
    return None
  return cycles[sorted_list[0][0]]


In [ ]:
def find_repcycle2(segment_wav, start_sample, f0, cycles):
  #score the cycles based on they adherence to the following biases:
  # TODO make algorithm to ignore noisy sections of the signal
  # 1. obvious shape at the beginning of the cycle
  # 2. closeness to f0
  # 3. loudness
  # 4. most in the middle
  
  if len(cycles) < 2:
    return cycles[0]  #if there is only one cycle return it
  
  mid_segment = start_sample+ (SEGMENT_LENGTH/2);
  f0_length = 1.0/f0 * AUDIO_SIZE #length of samples of fundamental frequency
  
  cycles_data = [] # (index, front, freq_score, rms_score, middle_score)
  for index, cycle in enumerate(cycles):
    
    i_beginning = math.floor(cycle[0]-start_sample)
    i_end = math.floor(cycle[1]-start_sample)
    cyc_wav = segment_wav[i_beginning:i_end]

    cyc_vec = vectorize_f(cyc_wav, FRONT_VEC_SIZE)
    front = bias_front_abs(cyc_vec)

    #get length of the cycle and comapre with fundamental frequency reciprocal 
    freq = cycle[1] - cycle[0]
    f_score = abs(freq - f0_length)
    
    #get middle and comare with middle of the whole segment  
    middle = (cycle[0] + cycle[1]) / 2
    m_score = abs(middle - mid_segment)

    #get rms
    rms = calculate_rms(cyc_wav)
    r_score = 1-rms

    #identify cycles that have an obvious shape at the beginning of the cycle by using bias_front

    #cyc_wavs.append(cyc_wav)
    cycles_data.append((index, front, f_score, r_score, m_score))

    
    # There seems to ba a positive correlation between the number of zero crossings and the fundamental frequency 
    # As the fundamental frequency increases because of either a higher pitch or more overtones in the upper register
    # There will be more zerocrossings. This can be intuitively observed as raising the pitch of a sine wave is equivalent
    # to squishing the waveform in-place and therefore increasing the number of zero crossings.
    # we can apply different cases of filtering for the most representative cycle by looking at the number of zero crossings
    
    # - For little zero crossing there is no need to find a majority, as we dont want to restrict our search
    # - A search for a majority comes in handy for medium number of zero crossings which can be indicitive of a steady pitch
    # - For a signal with many zero crossing (many overtones), a majority can be misleading and instead we should look for oultiers 
    # as they will have a larger front_bias

  if(len(cycles_data) <= 4): #No need to distinguish outliers if there are only 4 or less cycles
    filtered_list = cycles_data

  elif(len(cycles_data) > SEGMENT_LENGTH//50): # outliers may be more representative of the signal
    majority, outliers, mean = distinguish_outliers_by_front(cycles_data, Z_THRESHOLD)
    higher_outliers = [tup for tup in outliers if tup[1] > mean] 
    if higher_outliers:
      filtered_list = higher_outliers  
    else:
      filtered_list = majority
  else: 
    majority, outliers, mean = distinguish_outliers_by_front(cycles_data, Z_THRESHOLD)
    filtered_list = majority
  

    # Normalize scores
  max_f_score = max(filtered_list, key=lambda x: x[2])[2]
  max_r_score = max(filtered_list, key=lambda x: x[3])[3]
  max_m_score = max(filtered_list, key=lambda x: x[4])[4]

  normalized_cycles_data = [
    (index, front, f_score / max_f_score, r_score / max_r_score, m_score / max_m_score)
    for index, front, f_score, r_score, m_score in filtered_list
  ]

  # Apply weights and sort
  weighted_cycles_data = [
    (index, front, f_score / FREQ_WEIGHT, r_score / RMS_WEIGHT, m_score / MID_WEIGHT)
    for index, front, f_score, r_score, m_score in normalized_cycles_data
  ]
  
  sorted_list = sorted(weighted_cycles_data, key=lambda x: x[2]+x[3]+x[4])
  assert sorted_list != [], f"No repcycles found at {start_sample}"
  return cycles[sorted_list[0][0]]


In [ ]:
def find_repcycle3(segment_wav, start_sample, f0, cycles):
  #score the cycles based an obvious shape at the beginning of the cycle
  
  if len(cycles) < 2:
    return cycles[0]  #if there is only one cycle return it
  
  mid_segment = start_sample+ (SEGMENT_LENGTH/2);
  f0_length = 1.0/f0 * AUDIO_SIZE #length of samples of fundamental frequency

  cycles_data = [] # (index, front, front_abs, f_score, m_score)
  for index, cycle in enumerate(cycles):
    
    i_beginning = math.floor(cycle[0]-start_sample)
    i_end = math.floor(cycle[1]-start_sample)
    cyc_wav = segment_wav[i_beginning:i_end]

    #identify cycles that have an obvious shape at the beginning of the cycle 
    cyc_vec = vectorize_f(cyc_wav, FRONT_VEC_SIZE)
    front = bias_front_abs(cyc_vec)

    #get length of the cycle and comapre with fundamental frequency reciprocal 
    freq = cycle[1] - cycle[0]
    f_score = abs(freq - f0_length)

    #get middle and comare with middle of the whole segment  
    middle = (cycle[0] + cycle[1]) / 2
    m_score = abs(middle - mid_segment)

    cycles_data.append((index, front, f_score, m_score))

  if(len(cycles_data) <= 4): #No need to distinguish outliers if there are only 4 or less cycles
    candidates_list = cycles_data
  else:
    front_mean =  np.mean([tup[1] for tup in cycles_data])
    filtered_list = list(filter(lambda x, front_mean_abs=front_mean: x[1] > front_mean_abs, cycles_data))

    if len(filtered_list) > 1:
      candidates_list = filtered_list
    else:
      candidates_list = cycles_data

  assert candidates_list != [], f"No repcycles found at {start_sample} to {start_sample+SEGMENT_LENGTH}"

    # Normalize scores
  max_f_score = max(candidates_list, key=lambda x: x[2])[2]
  max_m_score = max(candidates_list, key=lambda x: x[3])[3]

  normalized_cycles_data = [
    (index, front, f_score / max_f_score, m_score / max_m_score)
    for index, front,  f_score, m_score in candidates_list
  ]

  # Apply weights and sort
  weighted_cycles_data = [
    (index, front,  f_score / FREQ_WEIGHT, m_score / MID_WEIGHT)
    for index, front,  f_score, m_score in normalized_cycles_data
  ]
  
  sorted_list = sorted(weighted_cycles_data, key=lambda x: x[2]+x[3])
  assert sorted_list != [], f"No repcycles found at {start_sample} to {start_sample+SEGMENT_LENGTH}"
  return cycles[sorted_list[0][0]]


In [ ]:
#waveform, sample_rate = torchaudio.load(AUDIO_PATH + FILE_NAME)
#waveform_arr = waveform.t().squeeze(1).numpy()
#
#split_waveform = np.array_split(waveform_arr, np.arange(SEGMENT_LENGTH, AUDIO_SIZE, SEGMENT_LENGTH))
#semgent_wav = split_waveform[SEGMENT_NUM]
##wav_vector = vectorize_f(semgent_wav,80)
#
##wav_vector = bias_front(split_waveform[SEGMENT_NUM],40)
##1show_waveform(semgent_wav, SEGMENT_NUM*SEGMENT_LENGTH)
##1show_waveform(wav_vector, SEGMENT_NUM*SEGMENT_LENGTH, filename=f"vectorized{SEGMENT_NUM*SEGMENT_LENGTH}")
#
#front = bias_front(semgent_wav)
#print(front)



### One Segment Test

In [36]:
waveform, sample_rate = torchaudio.load(AUDIO_PATH + FILE_NAME)
waveform_arr = waveform.t().squeeze(1).numpy()

# do not try to comprehend this....... I myself stopped trying to. This is an attempt to make the the algoithm automatically filter out noise 
# Signal-to-Noise Ratio or rather it's reciprocal Noise-to-Signal Ratio (NSR) is unique to a audio file and can be used to figure out how noisy it is
# NSR by itself is too low of a value to use as a filter hence the magic number NSR_FACTOR
# also it doesn't work............ hence the deprecation

#identify SNR reciprocal to get a dynamic noise thresholds for filtering 
rms_values, _ = rms_over_windows(waveform_arr, SEGMENT_LENGTH) #calculate RMS for each segment

# Get Noise-to-Signal Ratio
#signal_rms = np.max(rms_values)
#noise_rms = np.mean(rms_values)
#nsr = noise_rms/ signal_rms

# Calculate dynamic silence threshold based on NSR
#silence_1threshold = min(nsr * NSR_FACTOR, signal_rms*.50)

# Get Noise-to-Signal Ratio
signal_rms = np.max(rms_values)
noise_rms = np.mean(rms_values) * NOISE_FACTOR

# Calculate dynamic silence threshold based on NSR
silence_threshold = min(noise_rms, signal_rms*.50)

# Split the waveform into segments
split_waveform = np.array_split(waveform_arr, np.arange(SEGMENT_LENGTH, AUDIO_SIZE, SEGMENT_LENGTH))
#split_waveform = [waveform_np[i:i+segment_length] for i in range(0, 16000, segment_length)]

####### TESTING - get one segment ############
segment_wav = split_waveform[SEGMENT_NUM]
start_sample = SEGMENT_NUM*SEGMENT_LENGTH

# Get the Zero-Crossings, ignoring noise
zero_crossings = find_zerocrossings(segment_wav, start_sample, silence_threshold)
if len(zero_crossings) < 2:
  print(f"No zero crossings were found for segment [{start_sample}, {start_sample+SEGMENT_LENGTH}]")

# Get the fundamental frequency using the FFT
fft_mag, max_index = fft_max(segment_wav)

# exclude bins outside the F0_FREQ_RANGE
top_bins = klargest_with_indices(fft_mag, 4, (LOW_F0_INDEX,HIGH_F0_INDEX))
#itop_in_range = list(filter(lambda x, h=LOW_F0, l = HIGH_F0: l <= x[0] <= h, top_bins))[0]

# get frequency to bin index
# Fundamental Frequency: Sample Rate / fftsize * fft_bin_index
f0 = AUDIO_SIZE/FFT_SIZE * top_bins[0][0]

if len(top_bins) == 0:
  print(f"Failed to find a fundamental frequency in range ({F0_FREQ_RANGE[0]}Hz ,{F0_FREQ_RANGE[1]}Hz)")
#else:
  #print(f"f0: {f0}")
#print(f"silence_threshold: {silence_threshold}")
#print(f"F0 magnitude {fft_mag[max_index]}")

#find cycles within the signal and find a representative one for the segment

cycles = find_cycles_f0(f0, start_sample, zero_crossings)
if len(cycles) == 0: 
  print(f"Error No cycles were found for segment [{start_sample}, {start_sample+SEGMENT_LENGTH}]")

repcycle = None
if len(cycles) > 0:
  repcycle = find_repcycle3(segment_wav, start_sample, f0, cycles)
  if repcycle == None:
    print(f"Error: failed to find a representative cycle for segment [{start_sample}, {start_sample+SEGMENT_LENGTH}]")

show_waveform(segment_wav, start_sample, zero_crossings, repcycle)
show_fft(fft_mag, start_sample, 4)


No zero crossings were found for segment [9500, 10000]
Error No cycles were found for segment [9500, 10000]


### Main

In [39]:

waveform, sample_rate = torchaudio.load(AUDIO_PATH + FILE_NAME)
waveform_arr = waveform.t().squeeze(1).numpy()

#identify SNR reciprocal to get a dynamic noise threshold for filtering 
rms_values, _ = rms_over_windows(waveform_arr, SEGMENT_LENGTH) #calculate RMS over time

# Get Noise-to-Signal Ratio
#signal_rms = np.max(rms_values)
#noise_rms = np.mean(rms_values)
#nsr = noise_rms/ signal_rms

# Calculate dynamic silence threshold based on NSR
#silence_1threshold = min(nsr * NSR_FACTOR, signal_rms*.50)

# Get Noise-to-Signal Ratio
signal_rms = np.max(rms_values)
noise_rms = np.mean(rms_values) * NOISE_FACTOR

# Calculate dynamic silence threshold based on NSR
silence_threshold = min(noise_rms, signal_rms*.50)

# Split the waveform into segments. fortunately this also pads the file if it was too short to begin with - serendipity 
split_waveform = np.array_split(waveform_arr, np.arange(SEGMENT_LENGTH, AUDIO_SIZE, SEGMENT_LENGTH))
#split_waveform = [waveform_np[i:i+segment_length] for i in range(0, 16000, segment_length)]

repcycles = []

for segment_num, segment_wav in enumerate(split_waveform):
  #segment_wav = split_waveform[SEGMENT_NUM]s
  start_sample = segment_num*SEGMENT_LENGTH
  # Get the Zero-Crossings, ignoring noise
  zero_crossings = find_zerocrossings(segment_wav, start_sample, silence_threshold)
  if not len(zero_crossings) > 1:
    repcycles.append([])
  # Get the fundamental frequency using the FFT
    continue
  fft_mag, max_index = fft_max(segment_wav)

  # exclude bins outside the F0_FREQ_RANGE
  # get frequency to bin index

  top_bins = klargest_with_indices(fft_mag, 4, (LOW_F0_INDEX,HIGH_F0_INDEX))
  assert len(top_bins) > 0, f"Failed to find a fundamental frequency in range ({F0_FREQ_RANGE[0]}Hz ,{F0_FREQ_RANGE[1]}Hz)"
  #top_in_range = list(filter(lambda x, h=LOW_F0, l = HIGH_F0: l <= x[0] <= h, top_bins))[0]

  # Fundamental Frequency: Sample Rate / fftsize * fft_bin_index
  f0 = AUDIO_SIZE/FFT_SIZE * top_bins[0][0]

  #print(f"silence_threshold: {silence_threshold}")
  #print(f"f0: {f0}")
  #print(f"F0 magnitude{fft_mag[max_index]}")

  #find cycles within the signal and find a representative one for the segment
  cycles = find_cycles_f0(f0, start_sample, zero_crossings)
  if not len(cycles) > 0:
    repcycles.append([])
    continue

  repcycle = find_repcycle3(segment_wav, start_sample, f0, cycles)
  assert repcycle != None, f"Error: failed to find a representative cycle for segment [{start_sample}, {start_sample+SEGMENT_LENGTH}]"

  repcycles.append(repcycle)
  #show_waveform(segment_wav, start_sample, zero_crossings, repcycle)
  #show_fft(fft_mag, start_sample, 4)

show_repcycles(repcycles, waveform_arr, vectorize=True)

#for num in cycles:
#  print(f"({num[0]:.2f}, {num[1]:.2f})")
#with open('fft_freq.txt', 'w') as file:
#  for index, value in enumerate(fft_mag):
#    file.write(f"{value} at {index*AUDIO_SIZE/FFT_SIZE}")

#### TEST- output repcycle to text ####
#with open(f'out\{FILE_NAME.split('.')[0]}_repcycles.txt', 'w') as file:
#  for index, value in enumerate(repcycles):
#    if value:
#      file.write("%.3f, %.3f\n" % (value[0], value[1]))
#    else:
#      file.write("0\n")
#################################################################

#### TEST- output repcycle AUDIO ####
#for index, curr_repcycle in enumerate(repcycles):
#  if curr_repcycle:
#    repc_start = math.floor(curr_repcycle[0])
#    repc_end = math.floor(curr_repcycle[1])
#    rep_cycle_wav = waveform_arr[repc_start:repc_end]    
#    outfile_tensor = torch.tensor(rep_cycle_wav).unsqueeze(0)
#    torchaudio.save(f"out\{FILE_NAME.split(".")[0]}_{index*SEGMENT_LENGTH}.wav", outfile_tensor, 16000)
#################################################################

#### TEST- output audio 10 copies of repcycle ####
outfile_wav = []
for curr_repcycle in repcycles:
  if curr_repcycle:
    repc_start = math.floor(curr_repcycle[0])
    repc_end = math.floor(curr_repcycle[1])
    repcycle_wav = waveform_arr[repc_start:repc_end]
    #repcycle_wav_v = vectorize_f(repcycle_wav, REPC_VEC_SIZE)
    for _ in range(10):
      outfile_wav.extend(repcycle_wav)
  #else:
  #  outfile_wav.extend(np.zeros(REPC_VEC_SIZE))  # pad with silence if no repcycle found
    
outfile_tensor = torch.tensor(outfile_wav).unsqueeze(0)
torchaudio.save(f"out\{FILE_NAME.split(".")[0]}_size{SEGMENT_LENGTH}out.wav", outfile_tensor, 16000)
################################################################



<string>:107: SyntaxWarning: invalid escape sequence '\{'
<>:107: SyntaxWarning: invalid escape sequence '\{'
<string>:107: SyntaxWarning: invalid escape sequence '\{'
<>:107: SyntaxWarning: invalid escape sequence '\{'
C:\Users\Andrei\AppData\Local\Temp\ipykernel_2116\747639225.py:107: SyntaxWarning: invalid escape sequence '\{'
  torchaudio.save(f"out\{FILE_NAME.split(".")[0]}_size{SEGMENT_LENGTH}out.wav", outfile_tensor, 16000)
